### IMPORTS

In [ ]:
import ealib

import logging
import pandas as pd

In [ ]:
# EDGAR API requires request header
req_header = {"User-Agent": "roberto.brera.24@outlook.com"}

### Logging tests/setup

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)
logging.info("logging info")

### COMP TICKERS

In [ ]:
# Dictionary to pandas dataframe
tickers_df = ealib.get_tickers_df(req_header)
tickers_df.head()

In [ ]:
# Examples on searches performable on this dataframe
tickers_df.iloc[133]
tickers_df[tickers_df["ticker"] == "NVDA"]
tickers_df[0:4]

In [ ]:
# Search func examples
ealib.find_ticker(tickers_df, "NVDA")["cik_str"].iloc[0]

# Random companies
tickers_df.iloc[97:111]

In [ ]:
ealib.find_title_substring(tickers_df, "morgan")

### COMP METADATA

In [ ]:
# Find cik for some company given name substring
query_substr =  "gamestop"
query_ticker = ealib.find_title_substring(tickers_df, query_substr)
query_ticker

### Scraping select document types for a company in the tickers dataframe

In [ ]:
# Main parameter setting and fucntion call
comp_out_df = ealib.download_select_filings(
    req_header=req_header, 
    mrps = 8, 
    tickers_df = tickers_df.iloc[:1000], 
    root_dir = f"Recent filings", 
    query_forms = ["424B5", "S-3"], 
    max_days = 180, 
    write_txt=False, 
    write_pdf=True
)

comp_out_df

### COMPANY FACTS

In [ ]:
comp_facts = ealib.get_response_dict(ealib.companyfacts_url(query_cik), req_header, mrps=mrps)
us_gaap_facts_dict = comp_facts["facts"]["us-gaap"]
us_gaap_facts_dict.keys()

In [ ]:
comp_facts["facts"]["dei"].keys()

In [ ]:
"""
Shares outstanding
"""
pd.DataFrame(comp_facts["facts"]["dei"]["EntityCommonStockSharesOutstanding"]["units"]["shares"])

In [ ]:

"""
Market cap (proxy)
"""
pd.DataFrame(comp_facts["facts"]["dei"]["EntityPublicFloat"]["units"]["USD"])

In [ ]:

test_qr_substr = "stock"
filtered_keys = find_dict_key_substr(us_gaap_facts_dict, test_qr_substr)

"""
sub_dict = {key: us_gaap_facts_dict[key] for key in filtered_keys if key in us_gaap_facts_dict}
"""
filtered_keys

In [ ]:
us_gaap_facts_dict["CommonStockValue"]["units"]["USD"]

In [ ]:
# Convert to pd data frame
pd.DataFrame(us_gaap_facts_dict["CommonStockValue"]["units"]["USD"])

### COMPANY CONCEPT

In [ ]:
rev_concept = ealib.get_response_dict(
    ealib.companyconcept_url(query_cik, "/us-gaap/Revenues"), 
    req_header,
    mrps=mrps
)

# Coincides with company facts request
pd.DataFrame(rev_concept["units"]["USD"])